In [1]:
import findspark 
findspark.init("/Users/valentinaporcu/spark/spark-2.4.1-bin-hadoop2.7")
import pyspark 
from pyspark.sql import DataFrameNaFunctions 
from pyspark.sql.functions import lit 
from pyspark.ml.feature import StringIndexer  
from pyspark.ml import Pipeline 
from pyspark.sql import SparkSession
from pyspark.sql import functions
import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder.appName("nlp").getOrCreate()

In [3]:
df = spark.read.csv('/Users/valentinaporcu/Dropbox/topic/12.\ Guida\ ai\ Big\ Data\ con\ Python/codice\ -\ guida\ ai\ big\ data\ con\ Python/Sezione\ 3/churn.csv',
                      inferSchema=True,
                      header = True)

In [4]:
df.show()

+---+------+---+-----------+---------+-----+
|_c0|gender|age|  paymethod|lasttrans|label|
+---+------+---+-----------+---------+-----+
|565|  male| 32|credit card|       86|loyal|
|846|female| 44|credit card|       89|churn|
|666|  male| 70|       cash|      123|loyal|
|970|female| 21|credit card|      102|loyal|
|181|female| 53|credit card|      142|   NA|
|987|  male| 81|       cash|      153|   NA|
|789|female| 63|       cash|       89|loyal|
|  6|female| 47|       cash|      142|loyal|
| 47|female| 61|   bancomat|      117|churn|
|929|  male| 19|   bancomat|      108|loyal|
|570|female| 76|credit card|      162|churn|
|296|  male| 46|credit card|       95|   NA|
|798|  male| 53|credit card|       54|loyal|
|960|  male| 27|       cash|      130|   NA|
|748|  male| 27|credit card|       70|loyal|
|339|  male| 52|credit card|       77|loyal|
|125|  male| 34|credit card|      100|loyal|
|409|  male| 28|credit card|       43|loyal|
|790|female| 52|credit card|      107|loyal|
|712|femal

In [5]:
df.columns

['_c0', 'gender', 'age', 'paymethod', 'lasttrans', 'label']

In [ ]:
df.printSchema()

In [ ]:
df.na.drop(how ='any').show()

In [7]:
df = df.filter(df["label"] != 'NA')

In [ ]:
df.describe().show()

In [ ]:
# indexer = StringIndexer(inputCol= "gender", outputCol="genderIndex")

In [ ]:
from pyspark.ml import Pipeline

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in list(set(df.columns)-set(['_c0', 'age', 'lasttrans'])) ]

pipeline = Pipeline(stages=indexers)
indexed = pipeline.fit(df).transform(df)

indexed.show()

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols = ['age',
 'lasttrans',
 'gender_index',
 'paymethod_index'], outputCol = "features")

In [ ]:
output = assembler.transform(indexed)

In [ ]:
output.show()

In [ ]:
transformed_df = output.select('features','label_index')

In [ ]:
train, test = transformed_df.randomSplit([0.7,0.3])

In [ ]:
train.show()

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
nb = NaiveBayes(labelCol='label_index')

In [ ]:
nb_model = nb.fit(train)

In [ ]:
nb_predictions = nb_model.transform(test)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', 
                                          labelCol='label_index')

In [ ]:
nb_acc = evaluator.evaluate(nb_predictions)

In [ ]:
nb_predictions.show()

In [ ]:
nb_acc